Import libraries/set files

In [ ]:
import pandas as pd
import numpy as np
# re will be used to pull zip codes
import re

In [ ]:
# rename files
INTAKES = "Austin_Animal_Center_Intakes (1).csv"
OUTCOMES = "Austin_Animal_Center_Outcomes (1).csv"
STRAY = "Austin_Animal_Center_Stray_Map (1).csv"

Read in the data

In [ ]:
# read in the files
intakes = pd.read_csv(INTAKES)
outcomes = pd.read_csv(OUTCOMES)
stray = pd.read_csv(STRAY)

print("Rows loaded")
print("Intakes:", len(intakes))
print("Outcomes:", len(outcomes))
print("Stray:", len(stray))

Rows loaded
Intakes: 155815
Outcomes: 155784
Stray: 124


Cleaning the data

In [ ]:
# converting allows us to do time math such as the length of stay
intakes["IntakeDateTime"] = pd.to_datetime(intakes["DateTime"], errors="coerce")
outcomes["OutcomeDateTime"] = pd.to_datetime(outcomes["DateTime"],errors="coerce")
# errors="coerce" means that if a value can't be converted, it will become NA

/tmp/ipython-input-18697883.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  outcomes["OutcomeDateTime"] = pd.to_datetime(outcomes["DateTime"],errors="coerce")


In [ ]:
# drop the rows that are missing key values because if they are missing animal id or datetime, it cant be used
# dropna(subset) is what actaully recomes the rows missing these values
intakes = intakes.dropna(subset=["Animal ID", "IntakeDateTime"])
outcomes = outcomes.dropna(subset=["Animal ID", "OutcomeDateTime"])

print("After Cleaning:")
print("Intakes:", len(intakes))
print("Outcomes:", len(outcomes))

After Cleaning:
Intakes: 155815
Outcomes: 155784


In [ ]:
# i wanted to just double check that nothing was removed
print(intakes[["Animal ID", "IntakeDateTime"]].isna().sum())
print(outcomes[["Animal ID", "OutcomeDateTime"]].isna().sum())

Animal ID         0
IntakeDateTime    0
dtype: int64
Animal ID          0
OutcomeDateTime    0
dtype: int64


Key performance indicators

In [ ]:
# value_counts allows us to know how many of each outcome type we have
outcome_counts = outcomes["Outcome Type"].value_counts(dropna=True)
total_outcomes = outcomes["Outcome Type"].notna().sum()
adoptions = outcome_counts.get("Adoption", 0)
adoption_rate = adoptions / total_outcomes
# total outcomes is the number of rows that actually have an outcome type
# adoptions shows the number of adoptions
# the adoption rate is the number of adoptions divided by the total outcomes

print("Key Performance Indicators")
print("Total outcomes:", total_outcomes)
print("Total adoptions:", adoptions)
print("Overall adoption rate:", round(adoption_rate * 100, 2),"%percent")

print("Top outcome types (counts):")
print(outcome_counts.head(10))

Key Performance Indicators
Total outcomes: 155754
Total adoptions: 73304
Overall adoption rate: 47.06 %percent
Top outcome types (counts):
Outcome Type
Adoption           73304
Transfer           44768
Return to Owner    24363
Euthanasia          9932
Died                1466
Rto-Adopt           1053
Disposal             753
Missing               83
Relocate              27
Stolen                 5
Name: count, dtype: int64


In [ ]:
# we now need to figure out the adoption rate by different animal type
# use groupby and calculate the percent that are adoption outcomes
adoption_by_type = outcomes.groupby("Animal Type")["Outcome Type"].apply(
    lambda x: (x == "Adoption").mean()
).sort_values(ascending=False)

print("Adoption rate by Animal Type (in percent):")
print((adoption_by_type * 100).round(2))

Adoption rate by Animal Type (in percent):
Animal Type
Cat          49.67
Dog          48.83
Livestock    44.44
Bird         34.41
Other        10.32
Name: Outcome Type, dtype: float64


In [ ]:
# we need to estimate the length of stay using the first intake of the animals and first outcome for each animal id
first_intake_time = intakes.groupby("Animal ID")["IntakeDateTime"].min().reset_index()
first_outcome_time = outcomes.groupby("Animal ID")["OutcomeDateTime"].min().reset_index()
# merge the two together
los_table = pd.merge(
    first_intake_time,
    first_outcome_time,
    on="Animal ID",
    how="inner"
)
# find the length of stay in days
los_table["LengthOfStayDays"] = (
    los_table["OutcomeDateTime"] - los_table["IntakeDateTime"]
).dt.days

print("LENGTH OF STAY")
print("Number of animals with LOS:", len(los_table))
print("Median LOS (days):", los_table["LengthOfStayDays"].median())
print("Average LOS (days):", los_table["LengthOfStayDays"].mean())

LENGTH OF STAY
Number of animals with LOS: 138894
Median LOS (days): 5.0
Average LOS (days): 18.690663383587484


In [ ]:
# in order to analyze age/condition we need the intake info, so we will take one intake record per animal in the earliest intake row
intakes_sorted = intakes.sort_values(["Animal ID", "IntakeDateTime"])
first_intake_row = intakes_sorted.groupby("Animal ID").first().reset_index()
keep_cols = [
    "Animal ID", "IntakeDateTime",
    "Intake Type", "Intake Condition",
    "Sex upon Intake", "Age upon Intake",
    "Breed", "Color", "Found Location", "Animal Type"
]
first_intake_row = first_intake_row[keep_cols]
# now merge this into the los table
analysis_df = pd.merge(los_table, first_intake_row, on=["Animal ID", "IntakeDateTime"], how="left")

In [ ]:
# we cant do math with text (for example Age upon intake will product things like 1 week, 2 years), so we need to turn the text into numbers (days)
def age_to_days(age_str):
    if pd.isna(age_str):
        return np.nan
    text = str(age_str).lower()
    if "year" in text:
        return int(text.split()[0]) * 365
    if "month" in text:
        return int(text.split()[0]) * 30
    if "week" in text:
        return int(text.split()[0]) * 7
    if "day" in text:
        return int(text.split()[0])
    return np.nan

analysis_df["AgeDays"] = analysis_df["Age upon Intake"].apply(age_to_days)

analysis_df["AgeGroup"] = pd.cut(
    analysis_df["AgeDays"],
    bins=[-1, 180, 365, 3*365, 8*365, 100*365],
    labels=["<6mo", "6-12mo", "1-3y", "3-8y", "8y+"]
)

In [ ]:
# we now need adoption info which is in outcomes
# make a first outcome type table and merge it
first_outcome_type = outcomes.sort_values(
    ["Animal ID", "OutcomeDateTime"]
).groupby("Animal ID").first().reset_index()
first_outcome_type = first_outcome_type[["Animal ID", "Outcome Type"]]

# this keep only the columns we need
first_outcome_type = first_outcome_type[["Animal ID", "Outcome Type"]]

# we have to rename it so it doesn't collide with an existing Outcome Type column
first_outcome_type = first_outcome_type.rename(columns={"Outcome Type": "First Outcome Type"})
# remove the merge so we can re run the cell without any issues
if "First Outcome Type" in analysis_df.columns:
    analysis_df = analysis_df.drop(columns=["First Outcome Type"])

analysis_df = pd.merge(
    analysis_df,
    first_outcome_type,
    on="Animal ID",
    how="left"
)

adoption_by_age = analysis_df.groupby("AgeGroup")["First Outcome Type"].apply(
    lambda x: (x == "Adoption").mean()
)

print("ADOPTION RATE BY AGE GROUP (percent)")
print((adoption_by_age * 100).round(2))


ADOPTION RATE BY AGE GROUP (percent)
AgeGroup
<6mo      56.21
6-12mo    44.57
1-3y      38.28
3-8y      35.74
8y+       26.32
Name: First Outcome Type, dtype: float64


/tmp/ipython-input-1310649892.py:24: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adoption_by_age = analysis_df.groupby("AgeGroup")["First Outcome Type"].apply(


In [ ]:
# find the adoption rate by intake condition
adoption_by_condition = analysis_df.groupby("Intake Condition")["Outcome Type"].apply(
    lambda x: (x == "Adoption").mean()
).sort_values(ascending=False)

print("ADOPTION RATE BY INTAKE CONDITION (percent)")
print((adoption_by_condition * 100).round(2).head(50))

ADOPTION RATE BY INTAKE CONDITION (percent)
Intake Condition
Normal        50.14
Medical       48.55
Unknown       42.86
Other         39.88
Behavior      39.13
Pregnant      38.71
Nursing       29.64
Injured       28.89
Feral         27.07
Neonatal      24.55
Aged          22.15
Med Attn      20.00
Sick          19.60
Neurologic    12.50
Congenital     0.00
Agonal         0.00
Med Urgent     0.00
Panleuk        0.00
Parvo          0.00
Space          0.00
Name: Outcome Type, dtype: float64


In [ ]:
# I also want to use the zip codes to find animal hotspots
def extract_zip(location):

    if pd.isna(location):
        return np.nan

    text = str(location)

    parts = text.split()

    for p in parts:
        if len(p) == 5 and p.isdigit():
            return p

    return np.nan
# I am essentially turning location into text, splitting the text into words, and making sure that each word that has a length of 5 is treated as a zipcode
stray["zip"] = stray["Found Location"].apply(extract_zip)

austin_stray = stray[stray["zip"].str.startswith("787", na=False)]

top_zip = austin_stray["zip"].value_counts().head(50)

print("TOP STRAY ZIP HOTSPOTS")
print(top_zip)

TOP STRAY ZIP HOTSPOTS
zip
78745    11
78752     8
78703     8
78757     7
78753     6
78754     6
78741     6
78744     6
78723     4
78702     4
78721     4
78728     2
78704     2
78748     2
78701     2
78781     1
78725     1
78735     1
78746     1
Name: count, dtype: int64


In [ ]:
print("FINAL INSIGHTS")
print("- Overall adoption rate:", round(adoption_rate * 100, 2), "percent")

print("Best adoption animal types (percent):")
print((adoption_by_type.head(10) * 100).round(2))

print("Worst adoption animal types (percent):")
print((adoption_by_type.tail(5) * 100).round(2))

print("Median length of stay (days):",
      round(los_table["LengthOfStayDays"].median(), 2))

print("Top 5 stray ZIP hotspots:")
print(top_zip.head(10))

FINAL INSIGHTS
- Overall adoption rate: 47.06 percent
Best adoption animal types (percent):
Animal Type
Cat          49.67
Dog          48.83
Livestock    44.44
Bird         34.41
Other        10.32
Name: Outcome Type, dtype: float64
Worst adoption animal types (percent):
Animal Type
Cat          49.67
Dog          48.83
Livestock    44.44
Bird         34.41
Other        10.32
Name: Outcome Type, dtype: float64
Median length of stay (days): 5.0
Top 5 stray ZIP hotspots:
zip
78745    11
78752     8
78703     8
78757     7
78753     6
78754     6
78741     6
78744     6
78723     4
78702     4
Name: count, dtype: int64
